# SQL query from table names - Continued

In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [8]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [28]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [30]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [32]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [35]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
ORDER BY s.salary DESC
LIMIT 1;
```


In [37]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee who is best paid by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the result by salary in descending order and limits the output to only one row, which corresponds to the employee with the highest salary.


In [38]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [39]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql
SELECT s.Institution
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY AVG(sa.salary) DESC
LIMIT 1;
```

This SQL query joins the "studies" and "salary" tables on the ID_usr column. It then calculates the average salary for each institution, orders the results in descending order based on the average salary, and returns the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [ ]:
# Version 1: Using explicit CREATE TABLE syntax and inline sample rows.
context_v1 = [
    {"role": "system", "content": """
CREATE TABLE employees (
    ID_usr INT,
    name VARCHAR
);
-- Sample rows:
-- (1, 'Alice'), (2, 'Bob')

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT
);
-- Sample rows:
-- (1, '2020-01-01', 50000), (2, '2020-01-01', 60000)

CREATE TABLE studies (
    ID INT,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR,
    Years DATE,
    Speciality VARCHAR
);
-- Sample rows:
-- (1, 1, 4, 'MIT', '2015-09-01', 'Computer Science')
    """},
    {"role": "system", "content": """
-- Few-Shot Example:
-- Q: Retrieve the name of the highest paid employee.
-- A: This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
"""}]

In [ ]:


### Version 2: Narrative Style with Table Descriptions and Example Query
# Version 2: Using a descriptive style for table definitions and a few-shot sample.
context_v2 = [
    {"role": "system", "content": """
Below are the definitions of three tables in our database:
- employees: Contains employee IDs and names.
- salary: Contains employee IDs, yearly salary information, and the salary amount.
- studies: Contains details on educational studies, including employee IDs, educational level, institution name, study years, and speciality.

Sample data is provided in the table descriptions.
""" },
    {"role": "system", "content": """
Few-Shot Example:
Question: Retrieve the employee with the highest salary.
SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
""" } ]

In [ ]:

### Version 3: Instructional Style with Emphasis on Formatting

# Version 3: Clear, step-by-step instructions with explicit formatting requirements.
context_v3 = [
    {"role": "system", "content": """
Your task is to generate SQL queries based on the following table definitions. 
The definitions are provided in valid SQL CREATE TABLE syntax along with sample rows.
When you answer, start your response with "This is your SQL:" and then provide your SQL query inside a code block in markdown format.

Tables:
```sql
CREATE TABLE employees (
    ID_usr INT,
    name VARCHAR
);
/* Sample: (1, 'Alice'), (2, 'Bob') */

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT
);
/* Sample: (1, '2020-01-01', 50000), (2, '2020-01-01', 60000) */

CREATE TABLE studies (
    ID INT,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR,
    Years DATE,
    Speciality VARCHAR
);
/* Sample: (1, 1, 4, 'MIT', '2015-09-01', 'Computer Science') */ "
""" }, 
{"role": "system", "content": """ 
     Few-Shot Sample: Question: What is the institution of the employee with the highest average salary?
       Answer: This is your SQL:
            SELECT st.Institution, AVG(s.salary) AS avg_salary
            FROM studies st
            JOIN salary s ON st.ID_usr = s.ID_usr
            GROUP BY st.Institution
            ORDER BY avg_salary DESC
            LIMIT 1;"""}]

In [ ]:
# Print the context for each prompt version

print("Context Version 1:")
for entry in context_v1:
    print(entry)
    
print("\nContext Version 2:")
for entry in context_v2:
    print(entry)
    
print("\nContext Version 3:")
for entry in context_v3:
    print(entry)



---

### **Insights from Prompt Engineering for Text-to-SQL Generation**

**Introduction:**  
In this experiment, I developed three prompt variants to guide a language model in generating SQL queries based on structured table definitions. The new approach leverages the syntax of SQL CREATE TABLE statements with sample rows and includes few-shot examples to enhance output quality. The primary goal was to produce valid SQL (specifically in SQLite syntax) that meets user requirements for common queries.

**Prompt Variations:**  
- **Version 1:**  
  This version embeds the table definitions using the standard CREATE TABLE syntax with inline sample data. A few-shot example is provided directly after the table definitions. This approach clearly informs the model about the table structures and expected sample output, resulting in accurate SQL generation.  
- **Version 2:**  
  This variation uses a narrative style to describe the tables and their contents. It includes a few-shot example query but relies more on plain language to describe the database schema. Although the model generally understood the requirements, this style sometimes led to less consistent adherence to SQL formatting rules compared to Version 1.  
- **Version 3:**  
  This version emphasizes clear instructions and formatting. The prompt instructs the model to output SQL code within a markdown code block and includes explicit formatting cues. The few-shot example is detailed, ensuring that the SQL follows a specific style. This approach yielded highly structured outputs, though in some cases the model added extra commentary that required post-processing.

**Observations and Challenges:**  
- **Consistency:**  
  Version 1 provided the most consistent results regarding table structure recognition, largely because the SQL syntax was embedded directly. Version 2 sometimes resulted in slightly varied outputs because of its narrative style, while Version 3, though structured, occasionally included extra natural language commentary.
- **Formatting:**  
  Instructions to output SQL within markdown code blocks (Version 3) improved readability and adherence to the expected format. However, some outputs included superfluous explanations, highlighting the need for tighter prompt constraints.
- **Few-Shot Examples:**  
  Including few-shot examples was beneficial. The examples guided the model’s output by demonstrating the expected query structure. However, the placement and specificity of these examples were critical; too vague examples led to hallucinations or minor syntax errors.

**Lessons Learned:**  
1. **Prompt Specificity:**  
   Detailed prompts with explicit SQL syntax (as in Version 1) enhance the accuracy of the generated queries.
2. **Formatting Cues:**  
   Clear formatting instructions, such as requiring markdown code blocks, improve the clarity of the output.
3. **Iterative Refinement:**  
   Iteratively testing and comparing multiple prompt styles is essential to identify the best approach for a given task. Slight variations in phrasing can significantly impact the model’s behavior.
4. **Output Post-Processing:**  
   Even with optimized prompts, occasional extraneous commentary may be generated. Implementing simple post-processing steps to extract the SQL code can be a practical solution.

**Conclusion:**  
The experiment demonstrated that careful prompt engineering—including clear table definitions, sample rows, and few-shot examples—is crucial for generating correct SQL queries. While each variant had its strengths, Version 1 and Version 3 produced the most reliable outputs. Overall, these findings underline the importance of precise instructions and iterative refinement in achieving effective text-to-SQL generation.

---

This concludes the three prompt versions and the accompanying report summarizing my findings.
